# Predict low high models
Model selection for predict lo hi


To install conda and binance packages to this notebook uncomment the code below

In [1]:
#%conda install -c plotly plotly=5.9.0
#%conda install pip
#%conda install twisted
%pip install plotly==5.9.0
%pip install twisted
#%pip install binance-connector==1.13.0
%pip install pandas
%pip install scikit-learn
%pip install tensorflow==2.11.0
%pip install keras==2.11.0
%pip install scikeras


import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', None)

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable


Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


## Feature engineering
Read data from bot's history, resample to equal intervals, create X, y

In [2]:
import os
import glob
import pandas as pd
from datetime import timedelta,date

def read_last_data(strategy):
    """ Read last last day from bot history """
    
    def read_last(symbol: str, name: str):
        data_dir=f"./../data/{strategy}/Xy"
        file_path = sorted([f for f in os.listdir(data_dir) if f.endswith(f"{symbol}_{name}.csv")])[-1]
        return pd.read_csv(f"{data_dir}/{file_path}", parse_dates=True, index_col="datetime")

    data=read_last("BTCUSDT", "data")
    X=read_last("BTCUSDT", "X")
    y=read_last("BTCUSDT", "y")
    return data,X,y

#strategy="SimpleKerasStrategy"
strategy="LSTMStrategy"
bidask,X_bot,y_bot = read_last_data(strategy)
bidask

,symbol,bid,bid_vol,ask,ask_vol
datetime,,,,,
2023-05-05 02:23:56.214391,BTCUSDT,29261.63,4.61667,29261.64,0.18384
2023-05-05 02:23:56.840164,BTCUSDT,29261.63,4.61528,29261.64,0.18384
2023-05-05 02:23:56.937142,BTCUSDT,29261.63,4.61357,29261.64,0.18384
2023-05-05 02:23:57.034219,BTCUSDT,29261.63,4.61357,29261.64,0.17713
2023-05-05 02:23:57.130209,BTCUSDT,29261.63,4.61357,29265.81,0.36383
...,...,...,...,...,...
2023-05-05 02:25:55.947455,BTCUSDT,29257.12,5.00894,29257.13,0.00064
2023-05-05 02:25:56.147238,BTCUSDT,29257.12,5.30777,29257.13,0.00064
2023-05-05 02:25:56.308310,BTCUSDT,29257.12,6.18035,29257.13,0.00064


In [3]:

def get_targets(bidask):
        """ 
        Calculate targets - bid/ask bounds in future prediction window
        """
        predict_window="60s"
        fut_min=bidask[["bid", "ask"]][::-1].rolling(predict_window).min()[::-1].rename(columns={"bid":"bid_min_fut", "ask":"ask_min_fut"})
        fut_max=bidask[["bid", "ask"]][::-1].rolling(predict_window).max()[::-1].rename(columns={"bid":"bid_max_fut", "ask":"ask_max_fut"})
        return pd.concat([fut_min, fut_max], axis = 1)

y_raw = get_targets(bidask)
#pd.concat([X_bot,y_raw], axis=1).dropna()
dups=X_bot[X_bot.index.duplicated()]
dups.sort_index().tail(10)


,time_hour,time_minute,time_second,time_day_of_week,time_diff,spread,bid_diff,bid_vol_diff,ask_diff,ask_vol_diff,l2_bucket_-10,l2_bucket_-9,l2_bucket_-8,l2_bucket_-7,l2_bucket_-6,l2_bucket_-5,l2_bucket_-4,l2_bucket_-3,l2_bucket_-2,l2_bucket_-1,l2_bucket_0,l2_bucket_1,l2_bucket_2,l2_bucket_3,l2_bucket_4,l2_bucket_5,l2_bucket_6,l2_bucket_7,l2_bucket_8,l2_bucket_9
datetime,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2023-05-05 02:24:55.963987,2,24,55,4,175195000.0,0.01,0.0,0.03085,0.0,0.0,0.00082,0.0,0,0.0,0.0,0.0,0.02,0.0,0.00000,14.78115,14.03052,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-05-05 02:25:19.085349,2,25,19,4,97369000.0,0.01,0.0,0.02100,0.0,0.0,0.00082,0.0,0,0.0,0.0,0.0,0.00,0.0,0.00248,14.06251,6.45245,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-05-05 02:25:19.177356,2,25,19,4,92007000.0,0.01,0.0,0.13963,0.0,0.0,0.00082,0.0,0,0.0,0.0,0.0,0.00,0.0,0.00248,14.06251,6.45245,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-05-05 02:25:19.274069,2,25,19,4,96713000.0,0.01,0.0,0.03085,0.0,0.0,0.00082,0.0,0,0.0,0.0,0.0,0.00,0.0,0.00248,14.06251,6.45245,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-05-05 02:25:19.370637,2,25,19,4,96568000.0,0.01,0.0,0.00037,0.0,0.0,0.00082,0.0,0,0.0,0.0,0.0,0.00,0.0,0.00248,14.06251,6.45245,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-05-05 02:25:19.471391,2,25,19,4,100754000.0,0.01,0.0,0.05710,0.0,0.0,0.00082,0.0,0,0.0,0.0,0.0,0.00,0.0,0.00248,14.06251,6.45245,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-05-05 02:25:19.567399,2,25,19,4,96008000.0,0.01,0.0,0.34197,0.0,0.0,0.00082,0.0,0,0.0,0.0,0.0,0.00,0.0,0.00248,14.06251,6.45245,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-05-05 02:25:19.660309,2,25,19,4,92910000.0,0.01,0.0,0.02267,0.0,0.0,0.00082,0.0,0,0.0,0.0,0.0,0.00,0.0,0.00248,14.06251,6.45245,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-05-05 02:25:19.759723,2,25,19,4,99414000.0,0.01,0.0,0.07000,0.0,0.0,0.00082,0.0,0,0.0,0.0,0.0,0.00,0.0,0.00248,14.06251,6.45245,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Resample data to use equal time intervals

In [5]:
def resampled(X: pd.DataFrame, y: pd.DataFrame) -> (pd.DataFrame, pd.DataFrame):
    """ Resample to make equal intervals time series """
    time_interval="10s"
    Xy=pd.concat([X,y], axis=1)
    diffmap = [(c, "sum") for c in Xy.columns if c.endswith("diff")]
    timemap = [(c, "last") for c in Xy.columns if c.startswith("time") and not c.endswith("diff")]
    l2map = [(c, "last") for c in Xy.columns if c.startswith("l2_")]
    
    futmap = [(c, "last") for c in Xy.columns if c.endswith("_fut")]
    colmap = dict(diffmap + timemap + l2map + futmap + [("spread", "last")])
    resampled = Xy.resample(time_interval).agg(colmap).dropna()
    return resampled[X.columns], resampled[y.columns]
    
#X,y = resampled(X_bot, y_raw)
X,y=X_bot,y_raw
pd.concat([X,y], axis=1).dropna()

InvalidIndexError: Reindexing only valid with uniquely valued Index objects

## Create the model


In [ ]:
from keras import Sequential, Input
from keras.layers import Dense, Dropout
from keras.preprocessing.sequence import TimeseriesGenerator
from sklearn.compose import ColumnTransformer, TransformedTargetRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from scikeras.wrappers import KerasRegressor
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.compose import ColumnTransformer, make_column_selector


test_size=0.3
test_index = int(len(X)*(1-test_size))
X_train, y_train, X_test, y_test = X.iloc[:test_index], y.iloc[:test_index], X.iloc[test_index:], y.iloc[test_index:]

   
def create_model(X_train, y_train, window_size):
        
        model = Sequential()
        model.add(LSTM(128,  return_sequences=True, input_shape=(window_size, X_train.shape[1])))
        model.add(Dropout(0.2))
        model.add(LSTM(32))         
        model.add(Dropout(0.2))
        model.add(Dense(20, activation='relu'))
        model.add(Dense(y_train.shape[1], activation='linear'))
        #model.add(Dense(y_train.shape[1], activation='softmax'))
        model.compile(optimizer='adam', loss='mean_absolute_error', metrics=['mean_squared_error'])
        return model
    
    
x_scaler, y_scaler = StandardScaler().fit(X_train), StandardScaler().fit(y_train)
X_train = x_scaler.transform(X_train)
y_train = y_scaler.transform(y_train)

window_size = 10
gen = TimeseriesGenerator(X_train, y_train, length=window_size,
                               sampling_rate=1, batch_size=1)

model = create_model(X_train, y_train, window_size)
model.fit(gen,steps_per_epoch=1000, epochs=10)